In [105]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd
from bs4 import BeautifulSoup as bs

In [129]:
driver = webdriver.Chrome()
driver.get("http://info.nec.go.kr/")
time.sleep(0.5) # 페이지 로딩 대기

driver.find_element(By.XPATH, '//*[@id="menu"]/ul/li[3]/a').click() # 제22대 국회의원선거로 이동
time.sleep(0.5)

top_menu = driver.find_element(By.XPATH, '//*[@id="topmenu"]/nav/ul') # 호버할 메뉴 지정
ActionChains(driver).move_to_element(top_menu).perform() # ActionChains 사용하여 hover 동작 수행
time.sleep(0.1) # 호버 뜨는 시간 대기

driver.find_element(By.XPATH, '//*[@id="topmenu"]/nav/ul/li[2]/ul/li[2]/a').click() # 후보자 명단으로 이동
driver.find_element(By.XPATH, '//*[@id="electionId2"]').click() # 국회의원선거 선택

result = []

# 시도조회
driver.find_element(By.XPATH, '//*[@id="cityCode"]').click()
time.sleep(0.1)
city = driver.find_elements(By.XPATH, '//*[@id="cityCode"]/option')
city = city[1:] # 첫번째 option 제외

for i in range(len(city)):
    city = driver.find_elements(By.XPATH, '//*[@id="cityCode"]/option')
    city = city[1:] # 첫번째 option 제외
    city_option = city[i]
    city_option.click()
    time.sleep(0.5)

    # 선거구 드롭다운이 업데이트될 때까지 기다리기 (WebDriverWait 쓰는 법 익숙해지기)
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="sggCityCode"]/option[2]'))
    )
    
    # 선거구 조회
    district = driver.find_elements(By.XPATH, '//*[@id="sggCityCode"]/option')
    district = district[1:] # 첫번째 option 제외
    
    for j in range(len(district)):
        # 매번 새로 district 리스트 가져오기 위해 for문 내부에서 다시 가져옴
        district = driver.find_elements(By.XPATH, '//*[@id="sggCityCode"]/option')
        district = district[1:] # 첫번째 option 제외
        # 인덱스가 범위를 벗어나면 종료
        if j >= len(district):
            break
            
        district_option = district[j] # 현재 선택할 선거구 찾기
        district_option.click()
        time.sleep(0.5)

        driver.find_element(By.XPATH, '//*[@id="spanSubmit"]/input').click() # 검색 버튼 클릭
        time.sleep(1)

        # 결과 테이블에서 데이터 추출
        rows = driver.find_elements(By.XPATH, '//*[@id="table01"]/tbody/tr')
        for row in rows:
            num = row.find_element(By.XPATH, './td[3]').text
            party = row.find_element(By.XPATH, './td[4]').text
            name = row.find_element(By.XPATH, './td[5]').text
            gender = row.find_element(By.XPATH, './td[6]').text
            birth = row.find_element(By.XPATH, './td[7]').text.split()[0]
            age = row.find_element(By.XPATH, './td[7]').text.split()[1]
            address = row.find_element(By.XPATH, './td[8]').text
            job = row.find_element(By.XPATH, './td[10]').text
            education = row.find_element(By.XPATH, './td[10]').text
            career = row.find_element(By.XPATH, './td[11]').text
            result.append([num, party, name, gender, birth, age, address, job, education, career])

driver.quit() # 브라우저 닫기

df = pd.DataFrame(result, columns=['기호', '정당', '이름', '성별', '생년월일', '나이', '주소', '직업', '학력', '경력'])
print(df)

    기호       정당          이름 성별        생년월일     나이                     주소  \
0    1   더불어민주당  곽상언\n(郭相彦)  남  1971.11.18  (52세)          서울특별시 종로구 통일로   
1    2     국민의힘  최재형\n(崔在亨)  남  1956.09.02  (67세)         서울특별시 종로구 비봉5길   
2    6    새로운미래  진예찬\n(陳叡燦)  남  1978.12.20  (45세)       서울특별시 종로구 진흥로22길   
3    7     개혁신당  금태섭\n(琴泰燮)  남  1967.09.29  (56세)        서울특별시 종로구 혜화로6길   
4    8  가락특권폐지당  김준수\n(金俊洙)  남  1960.09.01  (63세)       서울특별시 종로구 수표로26길   
..  ..      ...         ... ..         ...    ...                    ...   
688  1   더불어민주당  김한규\n(金翰奎)  남  1974.07.01  (49세)        제주특별자치도 제주시 중앙로   
689  2     국민의힘  김승욱\n(金昇昱)  남  1967.09.30  (56세)  제주특별자치도 제주시 조천읍 중산간동로   
690  5    녹색정의당  강순아\n(康順娥)  여  1984.11.21  (39세)        제주특별자치도 제주시 태성로   
691  1   더불어민주당  위성곤\n(魏聖坤)  남  1968.01.20  (56세)       제주특별자치도 서귀포시 태평로   
692  2     국민의힘  고기철\n(高基喆)  남  1962.08.06  (61세)      제주특별자치도 서귀포시 흙담솔로   

                                                 직업  \
0                               

In [139]:
df.head()

,기호,정당,이름,성별,생년월일,나이,주소,직업,학력,경력
0,1,더불어민주당,곽상언\n(郭相彦),남,1971.11.18,(52세),서울특별시 종로구 통일로,서울대학교 국제경제학과 졸업,서울대학교 국제경제학과 졸업,(현) 더불어민주당 종로구 지역위원회 지역위원장\n(현) 법무법인 인강 대표변호사
1,2,국민의힘,최재형\n(崔在亨),남,1956.09.02,(67세),서울특별시 종로구 비봉5길,서울대학교 법학과 졸업,서울대학교 법학과 졸업,(현) 제21대 국회의원\n(전) 제24대 감사원장
2,6,새로운미래,진예찬\n(陳叡燦),남,1978.12.20,(45세),서울특별시 종로구 진흥로22길,프랑스 파리-발 드 센느 국립고등건축학교 건축학 석사(2010.09-2012.06),프랑스 파리-발 드 센느 국립고등건축학교 건축학 석사(2010.09-2012.06),(현) 건축사 (프랑스)\n(전) 민생당 최고위원
3,7,개혁신당,금태섭\n(琴泰燮),남,1967.09.29,(56세),서울특별시 종로구 혜화로6길,서울대학교 대학원 법학과 박사과정 수료,서울대학교 대학원 법학과 박사과정 수료,(전) 제20대 국회의원\n(현) 개혁신당 최고위원
4,8,가락특권폐지당,김준수\n(金俊洙),남,1960.09.01,(63세),서울특별시 종로구 수표로26길,부산대학교 사범대학 교육학과 2학년 제적(1980.03.~1986.08),부산대학교 사범대학 교육학과 2학년 제적(1980.03.~1986.08),(전) 제19대 국회의원선거 종로구 국민의힘 후보자


In [137]:
df.tail()

,기호,정당,이름,성별,생년월일,나이,주소,직업,학력,경력
688,1,더불어민주당,김한규\n(金翰奎),남,1974.07.01,(49세),제주특별자치도 제주시 중앙로,하버드대학교 로스쿨 법학석사(2011.8.~2012.5.),하버드대학교 로스쿨 법학석사(2011.8.~2012.5.),(현)제주시을 국회의원\n(전)대통령비서실 정무비서관
689,2,국민의힘,김승욱\n(金昇昱),남,1967.09.30,(56세),제주특별자치도 제주시 조천읍 중산간동로,서울대학교 대학원 건축학과 졸업(공학석사),서울대학교 대학원 건축학과 졸업(공학석사),(전)국민의힘 제주시을 당협위원장\n(현)사단법인 제주문화역사나들이 이사장
690,5,녹색정의당,강순아\n(康順娥),여,1984.11.21,(39세),제주특별자치도 제주시 태성로,제주대학교 가정관리학과 졸업,제주대학교 가정관리학과 졸업,(현)녹색정의당 제주시을위원장\n(현)동광초등학교 학교운영위원
691,1,더불어민주당,위성곤\n(魏聖坤),남,1968.01.20,(56세),제주특별자치도 서귀포시 태평로,제주대학교 농과대학 졸업,제주대학교 농과대학 졸업,(현) 국회의원(제20·21대)\n(전) 제주대학교 총학생회장
692,2,국민의힘,고기철\n(高基喆),남,1962.08.06,(61세),제주특별자치도 서귀포시 흙담솔로,숭실대학교 대학원 IT정책경영학과 졸업(공학박사),숭실대학교 대학원 IT정책경영학과 졸업(공학박사),(전)제주경찰청장\n(현)제주제2공항추진범시민포럼 명예대표


In [141]:
df.nunique()

기호        8
정당       21
이름      691
성별        2
생년월일    665
나이       50
주소      668
직업      610
학력      610
경력      692
dtype: int64

In [99]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd
from bs4 import BeautifulSoup as bs

# WebDriver 초기화
driver = webdriver.Chrome()
driver.get("http://info.nec.go.kr/")
time.sleep(0.5)  # 페이지 로딩 대기

# 제22대 국회의원선거로 이동
driver.find_element(By.XPATH, '//*[@id="menu"]/ul/li[3]/a').click()
time.sleep(0.5)

# 후보자 명단으로 이동
top_menu = driver.find_element(By.XPATH, '//*[@id="topmenu"]/nav/ul')
ActionChains(driver).move_to_element(top_menu).perform()
time.sleep(0.1)
driver.find_element(By.XPATH, '//*[@id="topmenu"]/nav/ul/li[2]/ul/li[2]/a').click()
time.sleep(0.5)

# 국회의원선거 선택
driver.find_element(By.XPATH, '//*[@id="electionId2"]').click()
time.sleep(0.5)

result = []

# 시도 조회
city_dropdown = driver.find_element(By.XPATH, '//*[@id="cityCode"]')
city_dropdown.click()
time.sleep(0.1)
city_options = driver.find_elements(By.XPATH, '//*[@id="cityCode"]/option')[1:]  # 첫번째 option 제외

for i in range(len(city_options)):
    try:
        # 매번 새로 city_options 리스트 가져오기
        city_options = driver.find_elements(By.XPATH, '//*[@id="cityCode"]/option')[1:]
        city_option = city_options[i]
        city_name = city_option.text  # 시도 이름 저장 (디버깅용)
        print(f"시도 선택: {city_name}")
        city_option.click()
        time.sleep(0.5)

        # 선거구 드롭다운이 업데이트될 때까지 기다리기
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="sggCityCode"]/option[2]'))
        )

        # 선거구 조회
        district_options = driver.find_elements(By.XPATH, '//*[@id="sggCityCode"]/option')[1:]  # 첫번째 option 제외
        
        # 선거구가 없는 경우 건너뛰기
        if not district_options:
            print(f"시도 {city_name}에는 선거구가 없습니다.")
            continue

        for j in range(len(district_options)):
            try:
                # 매번 새로 district_options 리스트 가져오기
                district_options = driver.find_elements(By.XPATH, '//*[@id="sggCityCode"]/option')[1:]
                if j >= len(district_options):  # 인덱스가 범위를 벗어나면 종료
                    break

                district_option = district_options[j]
                district_name = district_option.text  # 선거구 이름 저장 (디버깅용)
                print(f"선거구 선택: {district_name}")
                district_option.click()
                time.sleep(0.5)

                # 조회 버튼 클릭
                submit_button = driver.find_element(By.XPATH, '//*[@id="spanSubmit"]/input')
                submit_button.click()
                time.sleep(1)  # 결과 로딩 대기

                # 결과 테이블 가져오기
                res = driver.find_element(By.XPATH, '//*[@id="table01"]')
                res_txt = res.text
                result.append(res_txt)

            except Exception as e:
                print(f"선거구 {district_name}에서 오류 발생: {e}")
                continue

    except Exception as e:
        print(f"시도 {city_name}에서 오류 발생: {e}")
        continue

# 결과를 DataFrame으로 저장
df = pd.DataFrame(result, columns=["결과"])
print(df)

# WebDriver 종료
driver.quit()

시도 선택: 서울특별시
선거구 선택: 종로구
선거구 선택: 중구성동구갑
선거구 선택: 중구성동구을
선거구 선택: 용산구
선거구 선택: 광진구갑
선거구 선택: 광진구을
선거구 선택: 동대문구갑
선거구 선택: 동대문구을
선거구 선택: 중랑구갑
선거구 선택: 중랑구을
선거구 선택: 성북구갑
선거구 선택: 성북구을
선거구 선택: 강북구갑
선거구 선택: 강북구을
선거구 선택: 도봉구갑
선거구 선택: 도봉구을
선거구 선택: 노원구갑
선거구 선택: 노원구을
선거구 선택: 은평구갑
선거구 선택: 은평구을
선거구 선택: 서대문구갑
선거구 선택: 서대문구을
선거구 선택: 마포구갑
선거구 선택: 마포구을
선거구 선택: 양천구갑
선거구 선택: 양천구을
선거구 선택: 강서구갑
선거구 선택: 강서구을
선거구 선택: 강서구병
선거구 선택: 구로구갑
선거구 선택: 구로구을
선거구 선택: 금천구
선거구 선택: 영등포구갑
선거구 선택: 영등포구을
선거구 선택: 동작구갑
선거구 선택: 동작구을
선거구 선택: 관악구갑
선거구 선택: 관악구을
선거구 선택: 서초구갑
선거구 선택: 서초구을
선거구 선택: 강남구갑
선거구 선택: 강남구을
선거구 선택: 강남구병
선거구 선택: 송파구갑
선거구 선택: 송파구을
선거구 선택: 송파구병
선거구 선택: 강동구갑
선거구 선택: 강동구을
시도 선택: 부산광역시
선거구 선택: 중구영도구
선거구 선택: 서구동구
선거구 선택: 부산진구갑
선거구 선택: 부산진구을
선거구 선택: 동래구
선거구 선택: 남구
선거구 선택: 북구갑
선거구 선택: 북구을
선거구 선택: 해운대구갑
선거구 선택: 해운대구을
선거구 선택: 사하구갑
선거구 선택: 사하구을
선거구 선택: 금정구
선거구 선택: 강서구
선거구 선택: 연제구
선거구 선택: 수영구
선거구 선택: 사상구
선거구 선택: 기장군
시도 선택: 대구광역시
선거구 선택: 중구남구
선거구 선택: 동구군위군갑
선거구 선택: 동구군위군을
선거구 선택: 서구
선거구 선택: 북구갑
선거구 선택: 북구을
선거구 선택: 수성구갑
선거구 선택: 수성구